In [1]:
pip install --upgrade --quiet langchain langchain-community langchain-openai langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv llama-index py2neo tree_sitter json-repair sqlfluff jupyterlab ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
import re

from dotenv import load_dotenv

load_dotenv()


/Users/larissakemicloud.com/Library/Python/3.12/lib/python/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


True

In [3]:
os.environ["NEO4J_URI"] 
os.environ["NEO4J_USERNAME"]
os.environ["NEO4J_PASSWORD"]

# Agora você pode criar o gráfico sem passar os parâmetros diretamente
graph = Neo4jGraph()

In [4]:
# Função para limpar o código SQL
def clean_sql_code(sql_code):
    # Remover comentários de linha (--) e de bloco (/* ... */)
    sql_code = re.sub(r'(--[^\n]*)', '', sql_code)  # Remove comentários de linha
    sql_code = re.sub(r'(/\*[\s\S]*?\*/)', '', sql_code)  # Remove comentários de bloco
    
    # Remover espaços extras
    sql_code = re.sub(r'\s+', ' ', sql_code)  # Substitui múltiplos espaços por um único espaço
    sql_code = re.sub(r'\s*([,;()=<>])\s*', r'\1', sql_code)  # Remove espaços em torno de pontuação
    
    # Remover linhas vazias ou com espaços em branco
    sql_code = "\n".join([line for line in sql_code.splitlines() if line.strip()])
    
    return sql_code.strip()  # Remove espaços no início e no final

# Lista de caminhos de arquivos
file_paths = [
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoApiTokens.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoGrupo.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoGrupoUsuario.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoMenu.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoModuloTela.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoPermissao.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoPermissaoMenu.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoRecurso.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoTela.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoUsuario.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoUsuarioCliente.sql"
]

# Carregar os documentos
docs = []
for file_path in file_paths:
    loader = TextLoader(file_path, encoding="utf-8")
    docs.extend(loader.load())  # Adicionar documentos carregados à lista

# Limpar o código SQL antes de dividir
for doc in docs:
    doc.page_content = clean_sql_code(doc.page_content)

# Dividir os documentos em chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

# Verificar os documentos divididos
print(f"Total de documentos divididos: {len(documents)}")

# Exibir os primeiros chunks para verificar a limpeza
for idx, chunk in enumerate(documents[:3]):  # Mostrar os primeiros 3 chunks
    print(f"Chunk {idx + 1}:\n{chunk.page_content}\n")

Total de documentos divididos: 20
Chunk 1:
﻿CREATE TABLE [dbo].[ControleDeAcessoApiTokens]([ID] INT IDENTITY(1,1)NOT NULL,[ID_Empresa] INT NOT NULL,[ID_Usuario] INT NOT NULL,[Token] VARCHAR(500)NOT NULL,[TokenExpiration] DATETIME NULL,[DataDaUltimaAlteracao] DATETIME NOT NULL,[UltimoUsuarioQueAlterou] INT NOT NULL,CONSTRAINT [PK_ControleDeAcessoApiTokens] PRIMARY KEY CLUSTERED([ID] ASC),CONSTRAINT [FK_ControleDeAcessoApiTokens_ControleDeAcessoUsuario] FOREIGN KEY([ID_Usuario])REFERENCES [dbo].[ControleDeAcessoUsuario]([ID]),CONSTRAINT

Chunk 2:
[FK_ControleDeAcessoApiTokens_Empresa] FOREIGN KEY([ID_Empresa])REFERENCES [dbo].[Empresa]([ID]),CONSTRAINT [CK_ControleDeAcessoApiTokens_Token] UNIQUE NONCLUSTERED([Token] ASC),CONSTRAINT [CK_ControleDeAcessoApiTokens_Usuario] UNIQUE NONCLUSTERED([ID_Empresa] ASC,[ID_Usuario] ASC));

Chunk 3:
﻿CREATE TABLE [dbo].[ControleDeAcessoGrupo]([ID] INT IDENTITY(1,1)NOT NULL,[ID_Empresa] INT NOT NULL,[Nome] NVARCHAR(250)NOT NULL,[Descricao] NVARCHAR(600

In [8]:
documents[0]

Document(metadata={'source': '/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoApiTokens.sql'}, page_content='\ufeffCREATE TABLE [dbo].[ControleDeAcessoApiTokens]([ID] INT IDENTITY(1,1)NOT NULL,[ID_Empresa] INT NOT NULL,[ID_Usuario] INT NOT NULL,[Token] VARCHAR(500)NOT NULL,[TokenExpiration] DATETIME NULL,[DataDaUltimaAlteracao] DATETIME NOT NULL,[UltimoUsuarioQueAlterou] INT NOT NULL,CONSTRAINT [PK_ControleDeAcessoApiTokens] PRIMARY KEY CLUSTERED([ID] ASC),CONSTRAINT [FK_ControleDeAcessoApiTokens_ControleDeAcessoUsuario] FOREIGN KEY([ID_Usuario])REFERENCES [dbo].[ControleDeAcessoUsuario]([ID]),CONSTRAINT')

In [ ]:
from langchain.schema import Document

# Obter as chaves da API
openai_api_key = os.getenv("OPENAI_API_KEY")
llama_api_key = os.getenv("LLAMA_API_KEY")

# Definir qual LLM será utilizado
llm_type = os.getenv("LLM_TYPE", "openai")  # Valor padrão "openai" se não estiver definido

if llm_type == "ollama":
    # Usar o modelo LLaMA/Ollama e passar a chave de API da LLaMA
    llm = ChatOllama(model="llama3.1", temperature=0, api_key=llama_api_key)
else:
    # Usar o modelo OpenAI e passar a chave de API da OpenAI
    llm = ChatOpenAI(temperature=0, model="gpt-4o", api_key=openai_api_key)

# Criar o prompt especializado para SQL
def generate_sql_prompt(sql_code):
    return f"""
    Você está analisando um código SQL. Sua tarefa é identificar as seguintes informações:
    1. Tabelas (com nomes).
    2. Colunas em cada tabela.
    3. Chaves primárias e chaves estrangeiras.
    4. Relacionamentos entre tabelas.

    Aqui está o código SQL:
    {sql_code}

    Extraia essas informações de forma clara e concisa antes de transformá-las em um grafo.
    """

# Função para processar e transformar cada documento SQL em grafo
def process_sql_documents(documents, llm, llm_transformer):
    graph_documents = []

    for doc in documents:
        # Gerar o prompt especializado para SQL
        prompt = generate_sql_prompt(doc.page_content)
        
        # Montar a mensagem no formato correto para o LLM
        messages = [
            {"role": "system", "content": "Você é um assistente especializado em SQL."},
            {"role": "user", "content": prompt}
        ]
        
        # Usar o LLM diretamente para interpretar o SQL e extrair as informações
        extracted_info = llm(messages=messages)  # Passando a lista de mensagens
        
        # Acessar diretamente o conteúdo da resposta
        extracted_content = extracted_info.content
        
        # Criar um objeto Document a partir do conteúdo extraído
        document = Document(page_content=extracted_content)
        
        # Usar o LLMGraphTransformer para transformar em grafo
        graph_doc = llm_transformer.convert_to_graph_documents([document])
        graph_documents.extend(graph_doc)
    
    return graph_documents

# Usar o LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

# Converter documentos SQL para o formato de grafo, extraindo as informações importantes primeiro
graph_documents = process_sql_documents(documents, llm, llm_transformer)

# Verificar os primeiros documentos de grafo gerados
for idx, graph_doc in enumerate(graph_documents[:3]):
    print(f"Grafo {idx + 1}: {graph_doc}")

In [6]:
# Obter as chaves da API
openai_api_key = os.getenv("OPENAI_API_KEY")
llama_api_key = os.getenv("LLAMA_API_KEY")

# Definir qual LLM será utilizado
llm_type = os.getenv("LLM_TYPE", "openai")  # Valor padrão "openai" se não estiver definido

if llm_type == "ollama":
    # Usar o modelo LLaMA/Ollama e passar a chave de API da LLaMA
    llm = ChatOllama(model="llama3.1", temperature=0, api_key=llama_api_key)
else:
    # Usar o modelo OpenAI e passar a chave de API da OpenAI
    llm = ChatOpenAI(temperature=0, model="gpt-4o", api_key=openai_api_key)

# Criar o LLMGraphTransformer para extração de propriedades, nós e relacionamentos
llm_transformer_props = LLMGraphTransformer(
    llm=llm,
    # Definir tipos de nós permitidos, ajustados para extração de colunas e tabelas SQL
    allowed_nodes=["Table", "Column"],
    # Definir tipos de relacionamentos permitidos
    allowed_relationships=["HAS_COLUMN", "PRIMARY_KEY", "FOREIGN_KEY"],
    # Definir as propriedades que queremos extrair dos nós
    node_properties=["name", "type", "restrictions", "primary_key", "foreign_key", "unique"]
)

# Converter documentos SQL para o formato de grafo
graph_documents_props = llm_transformer_props.convert_to_graph_documents(documents)

# Verificar os primeiros nós e relacionamentos gerados
for idx, graph_doc in enumerate(graph_documents_props[:3]):
    print(f"Grafo {idx + 1}:")
    print(f"Nodes: {graph_doc.nodes}")
    print(f"Relationships: {graph_doc.relationships}")

Grafo 1:
Nodes: [Node(id='Controledeacessoapitokens', type='Table', properties={'name': 'ControleDeAcessoApiTokens'}), Node(id='Id', type='Column', properties={'name': 'ID', 'type': 'INT', 'restrictions': 'IDENTITY(1,1) NOT NULL', 'primary_key': 'true'}), Node(id='Id_Empresa', type='Column', properties={'name': 'ID_Empresa', 'type': 'INT', 'restrictions': 'NOT NULL'}), Node(id='Id_Usuario', type='Column', properties={'name': 'ID_Usuario', 'type': 'INT', 'restrictions': 'NOT NULL', 'foreign_key': 'ControleDeAcessoUsuario(ID)'}), Node(id='Token', type='Column', properties={'name': 'Token', 'type': 'VARCHAR(500)', 'restrictions': 'NOT NULL'}), Node(id='Tokenexpiration', type='Column', properties={'name': 'TokenExpiration', 'type': 'DATETIME', 'restrictions': 'NULL'}), Node(id='Datadaultimaalteracao', type='Column', properties={'name': 'DataDaUltimaAlteracao', 'type': 'DATETIME', 'restrictions': 'NOT NULL'}), Node(id='Ultimousuarioquealterou', type='Column', properties={'name': 'UltimoUsua

In [8]:
graph_documents_props[0]

GraphDocument(nodes=[Node(id='Controledeacessoapitokens', type='Table', properties={'name': 'ControleDeAcessoApiTokens'}), Node(id='Id', type='Column', properties={'name': 'ID', 'type': 'INT', 'restrictions': 'IDENTITY(1,1) NOT NULL', 'primary_key': 'true'}), Node(id='Id_Empresa', type='Column', properties={'name': 'ID_Empresa', 'type': 'INT', 'restrictions': 'NOT NULL'}), Node(id='Id_Usuario', type='Column', properties={'name': 'ID_Usuario', 'type': 'INT', 'restrictions': 'NOT NULL', 'foreign_key': 'ControleDeAcessoUsuario(ID)'}), Node(id='Token', type='Column', properties={'name': 'Token', 'type': 'VARCHAR(500)', 'restrictions': 'NOT NULL'}), Node(id='Tokenexpiration', type='Column', properties={'name': 'TokenExpiration', 'type': 'DATETIME', 'restrictions': 'NULL'}), Node(id='Datadaultimaalteracao', type='Column', properties={'name': 'DataDaUltimaAlteracao', 'type': 'DATETIME', 'restrictions': 'NOT NULL'}), Node(id='Ultimousuarioquealterou', type='Column', properties={'name': 'Ultimo

In [10]:
graph.add_graph_documents(
    graph_documents_props,
    baseEntityLabel=True,
    include_source=True
)

In [11]:
def showGraph():
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    
    session = driver.session()
    widget = GraphWidget(graph=session.run("MATCH (s)-[r:MENTIONS]->(t) RETURN s,r,t").graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [12]:
graph.query("SHOW INDEXES")

[{'id': 3,
  'name': 'constraint_907a464e',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'RANGE',
  'entityType': 'NODE',
  'labelsOrTypes': ['__Entity__'],
  'properties': ['id'],
  'indexProvider': 'range-1.0',
  'owningConstraint': 'constraint_907a464e',
  'lastRead': neo4j.time.DateTime(2024, 10, 4, 15, 8, 11, 516000000, tzinfo=<UTC>),
  'readCount': 2979},
 {'id': 1,
  'name': 'index_343aff4e',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'NODE',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 10, 4, 16, 36, 8, 160000000, tzinfo=<UTC>),
  'readCount': 2394},
 {'id': 2,
  'name': 'index_f7700477',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'RELATIONSHIP',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRea

In [13]:
# Cria uma vectorstore
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

In [14]:
schema_data = print(graph.schema)

Node properties:
Document {id: STRING, text: STRING, embedding: LIST}
Column {id: STRING}
Table {id: STRING}
Primary_key {id: STRING}
Foreign_key {id: STRING}
Relationship properties:

The relationships:
(:Document)-[:MENTIONS]->(:Table)
(:Document)-[:MENTIONS]->(:Column)
(:Document)-[:MENTIONS]->(:Foreign_key)
(:Document)-[:MENTIONS]->(:Primary_key)
(:Column)-[:PRIMARY_KEY]->(:Table)
(:Column)-[:REFERENCES]->(:Table)
(:Column)-[:REFERENCES]->(:Column)
(:Column)-[:FOREIGN_KEY]->(:Column)
(:Table)-[:HAS_COLUMN]->(:Column)
(:Table)-[:FOREIGN_KEY]->(:Table)
(:Table)-[:FOREIGN_KEY]->(:Column)
(:Table)-[:REFERENCES]->(:Table)
(:Table)-[:PRIMARY_KEY]->(:Column)
(:Table)-[:HAS_PRIMARY_KEY]->(:Primary_key)
(:Table)-[:HAS_FOREIGN_KEY]->(:Foreign_key)
(:Foreign_key)-[:REFERENCES]->(:Table)


In [15]:
name_tables = graph.query("MATCH (t:Table) RETURN t.id")

In [20]:
# Perguntas básicas sobre o Graph
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# Definir o template para geração de Cypher
CYPHER_GENERATION_TEMPLATE = """Objetivo: Gerar uma instrução Cypher para consultar o banco de dados Neo4J que possui uma estrutura de grafos representando um Schema de Banco de dados SQL.
Instruções:
- Verifique o esquema do banco de dados SQL consultando o Neo4j para obter os nomes das tabelas e colunas.
- Construa uma query Cypher para obter todas as Tabelas e Colunas que relacionem com o Prompt do usário.
- Certifique-se de verificar os relacionamentos (chaves estrangeiras) no banco de dados gráfico.
- Não inclua explicações ou desculpas em suas respostas. Apenas forneça a query SQL gerada.
- Não responda a perguntas que solicitem qualquer coisa além da construção de uma instrução Cypher.
- Não inclua nenhum texto além da instrução Cypher gerada.
----------
Task: Com base no prompt de usuário abaixo faça os seguintes passos:
1. Extraia metadados/termos do prompt do usuário que sejam importantes ou contenham possíveis nomes de entidades e colunas.
2. Busque na Relação de tabelas existentes no banco tabelas que possam satisfazer a query do usuário e utilize elas para formatar os termos utilizados na busca Cypher.
3. Gere uma query Chyper que consulte todas as tabelas ou colunas com esses nomes.
4. Considre consultas relativas (LIKE) em cima dos termos buscados.
----------
Exemplo de Query Cypher para ser utilizado:
 
MATCH (t:Table)-[:HAS_COLUMN]->(c:Column)
WHERE t.id CONTAINS 'Controledeacessorecurso' OR (c.id CONTAINS 'Id_Empresa')
RETURN t.id, c.id
----------
Relação do nome das tabelas existentes dentro do banco de dados Neo4j:
Controledeacessoapitokens
Controledeacessousuario
Empresa
Controledeacessogrupo
Controledeacessogrupousuario
Controledeacessomenu
Controledeacessomodulotela
Controledeacessomodulo
Controledeacessotela
Controledeacessopermissao
Controledeacessorecurso
Controledeacessopermissaomenu
Usuario
Controledeacessousuariocliente
 
----------
User prompt:
{query}

Schema banco de dados:
{schema}
"""

# Criar o template para o prompt
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "query"], template=CYPHER_GENERATION_TEMPLATE
)

# Conectar ao modelo da OpenAI
llm = ChatOpenAI(temperature=0)

# Conectar ao Neo4j e criar o chain
chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,  # Passa a conexão ao banco de dados Neo4j
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    allow_dangerous_requests=True,  # Permitir consultas perigosas
)

# Obter o esquema do banco de dados Neo4j automaticamente
schema_data = graph.schema  

# Definir a query
query = "Quais sao os usuarios ativos do sistema?"

# Executar a cadeia para gerar a consulta Cypher a partir do schema e da query
result = chain.run({"schema": schema_data, "query": query})

# Imprimir a consulta Cypher gerada
print(f"Cypher gerada: {result}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Table)-[:HAS_COLUMN]->(c:Column)
WHERE t.id CONTAINS 'Controledeacessousuario' OR t.id CONTAINS 'Usuario' OR t.id CONTAINS 'Controledeacessousuariocliente'
RETURN t.id, c.id
Full Context:
[{'t.id': 'Controledeacessousuario', 'c.id': 'Redefinirsenha'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Datadaultimaatualizacao'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Databloqueiousuario'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Email'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Datadecriacao'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Status'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Tipo'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Nomecompleto'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Identificacao'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Senha'}]

> Finished chain.
Cypher gerada: Eu não sei a resposta.
